# Randomisierte Algorithmen

Algorithmen, in denen der Zufall eine Rolle spielt, werden randomisiert, probabilistisch oder stochastisch genannt. Dem Einsteiger stellt sich dabei zunächst die Frage, wie der Zufall effektiv bei der Lösung eines Problems helfen kann. Der *"Zufallsregen"* zeigt das sehr eindrucksvoll.

## Bestimmung von $\pi$

Die Kreiszahl $\pi$ ist eine mathematische Konstante und definiert das Verhältnis zwischen Umfang $u$ und Durchmesser $d$ eines Kreises.

$$ \pi = \frac{u}{d} $$

Die Fläche $A$ eines Kreises ist definiert durch

$$ A = \frac{\pi}{4} d^2 $$

wobei $d^2$ nichts anderes als die Fläche des den Kreis umgebenden Quadrats ist.
Die Zahl $\pi$ kann also berechnet werden durch

$$ \pi = 4 \frac{A}{d^2} $$

wobei $\frac{A}{d^2}$ einfach das Verhältnis zwischen den Flächeninhalten von Kreis und Quadrat ist.

<img src="img/Rand_CircleQuad.png" width="200">

Zur Bestimmung dieses Verhältnisses kann man es nun zufällig Punkte (Koordinaten) *regnen lassen*. Im Bild oben sind z.B. von den insgesamt 14 *Tropfen* zufällig 11 innerhalb des Kreises gelandet. Da die Flächen im Verhältnis $\frac{\pi}{4}$ stehen, müssen zufällig positionierte Koordinaten auch in eben diesem Verhältnis auf den Flächen verteilt sein. Für das Beispiel ergibt sich:

$$ \frac{11}{14}*4 = 3,142857143... \approx \pi = 3,141592654... $$

Mit einer größeren Anzahl von Zufallstropfen sollte sich das Verhältnis stabilisieren und an $\pi$ annähern.

### Implementation

Im Folgenden wird nur ein Viertel des Einheitskreises und dessen umgebendes Quadrat betrachtet. Da beide Flächen um den gleichen Prozentsatz verkleinert werden, hat das keinen Einfluss auf das Verhältnis der Flächen zueinander. Da für x und y nur Gleitkommazahlen im Intervall $(0,1]$ generiert werden, liegen alle Koordinaten innerhalb des Quadrats. Für jede Koordinate $(x,y)$ innerhalb des Kreises gilt $x^2+y^2 \leq 1$ . So kann sehr einfach ermittelt werden, ob sich ein zufälliger Tropfen innerhalb des Viertelkreises befindet.

In [4]:
import random
import math

def calcPi():
    print("RealPi:",math.pi)
    cntCircle = 0
    cntRect = 0
    for i in range(1, 10000000):
        x = random.random()*-1+1
        y = random.random()*-1+1
        if x * x + y * y <= 1:
            cntCircle += 1
        cntRect += 1
        if i % 1000000 == 0:
            print("Approx:",4 * cntCircle / cntRect)

calcPi()

RealPi: 3.141592653589793
Approx: 3.138848
Approx: 3.14032
Approx: 3.139936
Approx: 3.140588
Approx: 3.1406928
Approx: 3.1407693333333335
Approx: 3.141190285714286
Approx: 3.1412935
Approx: 3.141352888888889


Bei genauer Betrachtung wird man feststellen, dass selbst wenn dieses Programm sehr lange weiter laufen würde, niemals exakt $\pi$ das Ergebnis wäre. Zum einen ist das darauf zurückzuführen, dass die Gleitkommazahlen mit denen hier gerechnet wird, nur eine begrenzte Genauigkeit aufweisen. Bei der Berechnung müssen intern also Rundungen durchgeführt werden, die zu einer Verfälschung des Ergebnisses führen. Zum anderen liegt die Ursache in der Berechnung der Zufallszahlen selbst.

## Zufallszahlen

Das Ergebnis unter Einbeziehung des Zufallsregens beruht im wesentlichen darauf, dass die generierten Zufallszahlen auch wirklich zufällig sind. In korrekt arbeitenden Computersystemen gibt es aber generell keinen echten Zufall (eine Ausnahme bilden System die physikalische Werte messen). Zurecht darf also die Frage gestellt werden, was das Ergebnis des Aufrufs von `random()` dann eigentlich ist.

Genau genommen sind diese Zahlen nämlich keine zufälligen Zahlen, denn sie werden durch einen deterministischen Algorithmus berechnet. Im Allgemeinen spricht man hier von **Pseudozufallszahlen**. Eine sehr einfacher Zufallszahlengenerator ist der **lineare Kongruenzgenerator**.

$$ z_n = (a \cdot z_{n-1}+b)\bmod c \text{ für } n\geq 1 $$

Die natürlichen Zahlen $a \in \{1,\dots, c-1\}$, $b \in \{0,\dots, c-1\}$ und $c \geq 2$ sind Konstanten, die die Güte des Generators beeinflussen. Die rekursive Notation weist darauf hin, dass die berechnete Zahl $z_n$ von der vorher berechneten Zahl $z_{n-1}$ abhängig ist. Ausgehend von einem beliebigen $z_0 \in \mathbb{N}$ kann hier also eine Folge berechnet werden, deren einzelne Glieder bei geeigneter Konstantenwahl in der Tat zufällig aussehen.

In [5]:
a = 9749
b = 0
c = 262144
z = 1234

def rand():
    global z
    z = (a * z + b) % c
    return z

print("0 :",z)
for i in range(1,10):
    print(i,":",rand())
for _ in range(c-10):
    rand()
print("...")
print(c,":",rand())
print(c+1,":",rand())
print(c+2,":",rand())

0 : 1234
1 : 233786
2 : 99778
3 : 181482
4 : 58162
5 : 3866
6 : 203042
7 : 7114
8 : 148370
9 : 210682
...
262144 : 1234
262145 : 233786
262146 : 99778


**Hintergrund:** *Die Wahl der Konstanten $a$, $b$ und $c$ ist entscheidend für die Güte des Zufallsgenerators. Die Konstante $c$ gibt an wieviele verschiedene Zufallszahlen überhaupt möglich sind. Die möglichen Zufallszahlen sind $[0,c)$. Ein weiteres Gütemerkmal ist die Periodenlänge, also die Anzahl der Berechnungen bis sich eine Zahl wiederholt. Diese kann maximiert werden indem ein $b$ gewählt wird, das zu $c$ teilerfremd ist, jeder Primfaktor von $c$ ein Teiler von $a-1$ ist und wenn $c$ durch 4 teilbar ist, auch $a-1$ durch 4 teilbar ist. Für die im Programmierbeispiel verwendeten Konstanten ist das der Fall.*

Wiederholtes Ausführen mit gleichem $z_0$ wird stets die gleiche Folge von Zahlen generieren. Aus diesem Grund wird bei Zufallsgeneratoren in regelmäßigen Abständen oder auch manuell ein neues $z_o$ generiert. Eine bewährte Praxis ist, die Systemzeit als sogenanntes **Seed** zu verwenden.

In [6]:
import time
z = int(round(time.time() * 1000))
print("0 :",z)
for i in range(1,10):
    print(i,":",rand())

0 : 1534602713494
1 : 89422
2 : 146278
3 : 862
4 : 15030
5 : 251118
6 : 248710
7 : 103934
8 : 66006
9 : 191118


## Las Vegas, Monte Carlo und Atlantic City

Diese für das Glückspiel (der Zufall lässt grüßen) bekannten Städte sind Namensgeber für ein paar bestimmte Typen von probabilistischen Algorithmen. 

*Las Vegas Algorithmen* liefern niemals falsche Ergebnisse und tun das mit einer gewissen Wahrscheinlichkeit auch schnell. Manchmal geraten sie in Sackgassen, sodass sei kein Resultat liefern. 

Im Gegenzug sind *Monte Carlo Algorithmen* immer schnell. Deren Ergebnisse sind aber möglicherweise falsch. 

*Atlantic City Algorithmen* sind ein Kompromiss dieser beiden Typen. Sie sind fast immer schnell und liefern fast immer ein korrektes Ergebnis. Da der Entwurf von Atlantic City Algorithmen sehr schwierig ist, gibt es nur wenige.

### Quicksort
Das typische Beispiel für einen **Las Vegas Algorithmus** erhält man durch Anpassung des Quicksort-Algorithmus. Normalerweise wird hier als Pivot-Element einfach das erste Element der zu sortierenden Liste verwendet. Ist die Liste aber bereits sortiert, führt dieses Verhalten zwangsläufig zur Worst-Case-Laufzeit von $\mathcal{O}(n^2)$. Wird das Pivot-Element dagegen zufällig ausgewählt, ändert sich etwas. Natürlich besteht weiterhin die Möglichkeit, dass zufällig immer das kleinste Element ausgewählt wird. Diese Wahrscheinlichkeit ist aber gegenüber der Wahrscheinlichkeit, dass eine Liste bereits sortiert ist, in der Praxis eher gering. Somit ist das Las-Vegas-Quicksort mit einer hohen Wahrscheinlichkeit schneller als das Standard-Quicksort.

In [1]:
import random
n = 0
def quicksortLasVegas(lst):
    global n
    n += 1
    if len(lst) < 2:
        return lst
    pivot = random.choice(lst) # zufällige Auswahl eines Elements
    #pivot = lst[0]
    smaller = []
    equal = []
    larger = []
    for i in range(0, len(lst)):
        if lst[i] < pivot:
            smaller.append(lst[i])
        elif lst[i] > pivot:
            larger.append(lst[i])
        else:
            equal.append(lst[i])
    return quicksortLasVegas(smaller) + equal + quicksortLasVegas(larger)

print(quicksortLasVegas([1, 2, 3, 4, 5, 7, 8]))
print("Calls:", n)

[1, 2, 3, 4, 5, 7, 8]
Calls: 9


### Äquivalenz zweier Listen

Üblicherweise prüft man zwei Listen darauf, ob sie die gleichen Elemente enthalten, indem diese zunächst sortiert und dann elementweise verglichen werden. Im Mittel benötigt eine solche Sortierung eine Laufzeit von $\mathcal{O}(n\;log\;n)$. Folgendes Beispiel verdeutlicht, dass die vollständige Prüfung $\mathcal{O}(n+2n\;log\;n)$ benötigt.

In [2]:
A = [1, 2, 2, 3, 4, 5, 7, 8]
B = [8, 2, 4, 1, 7, 5, 2, 3]
def containsSame(a, b):
    if len(a) != len(b):
        return False
    a = quicksortLasVegas(a)
    b = quicksortLasVegas(b)
    for i in range(len(a)):
        if a[i] != b[i]:
            return False
    return True
    
print(containsSame(A,B))

True


Ein **Monte Carlo Algorithmus** kann das sehr viel effizienter, jedoch mit der Einschränkung, dass zu einer gewissen Wahrscheinlichkeit das Ergebnis nicht stimmt. Dazu stelle man sich für die Liste A mit den Elementen $a_1$ bis $a_n$ und die Liste B mit den Elementen $b_1$ bis $b_n$ folgende Polynome vor:

$$
p_A(x) = (x-a_1)(x-a_2)\dots(x-a_n)\\
p_B(x) = (x-b_1)(x-b_2)\dots(x-b_n)
$$

Wenn jedes $a_i$ in B und jedes $b_i$ in A vorkommt, dann sind die Funktionen identisch. Durch Einsetzen eines beliebigen $x$ kann die Gleichheit der Funktionen geprüft werden. Wenn $p_A(x) \neq p_B(x)$, dann kann mit Sicherheit festgestellt werden, dass keine Äquivalenz der Listen vorliegt. Anderenfalls ist die Äquivalenz wahrscheinlich. Jedoch ist nicht ganz ausgeschlossen, dass trotz positiver Prüfung, keine Äquivalenz vorliegt. Folgendes Beispiel zeigt, dass ein falsches Ergebnis nicht nur dann produziert wird, wenn $x$ ein Element beider Listen ist (also bei beiden Funktionen eine Nullstelle getroffen wurde). 

$$
p_A(x) = (x-1)(x-2)(x-4)(x-5)\\
p_B(x) = (x-7)(x-2)(x-4)(x-2)\\
p_A(2) = 0 = p_B(2)\\
p_A(3) = 4 = p_B(3)
$$

Man kann aber die Wahrscheinlichkeit für die Korrektheit des Ergebnisses erhöhen, indem $x$ überlegt gewählt wird und/oder der Algorithmus mehrfach mit verschiedenen $x$ durchgeführt wird.

In [13]:
Y = [1,2,4,5]
Z = [7,2,4,2]
def containsSameMonteCarlo(a, b, maxX):
    if len(a) != len(b):
        return False
    x = random.randint(1,maxX)
    ax = 1
    bx = 1
    for i in range(len(a)):
        ax *= x-a[i]
        bx *= x-b[i]
    if ax != bx:
        return False
    return True
for _ in range(10):
    print(containsSameMonteCarlo(Y,Z,5))

True
True
False
True
False
True
True
False
True
True
